In [ ]:
import gc

# Uncomment below cell to use GPU for computation

In [ ]:
import tensorflow as tf

physical_device = tf.config.experimental.list_physical_devices('GPU')
print('No. of GPU: ', len(physical_device))
tf.config.experimental.set_memory_growth(physical_device[0], True)

# <span style = 'background: Red'> CNN for image classifiaction </span>

## Importing training dataset 

In [ ]:
train_folder =  'train'

In [ ]:
# pip install matplotlib

In [ ]:
import os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

## Viewing some random images from the training folder

In [ ]:
files = np.random.choice(os.listdir(train_folder), 6)

fig, axes = plt.subplots(2,3, figsize=(15, 7),
                         subplot_kw={'xticks': [], 'yticks': []})
    
axes = axes.flat

for file, ax in list(zip(files, axes)) :
    img_path = os.path.join(train_folder, file)
    img = mpimg.imread(img_path)
    ax.imshow(img)

## Loading images and labels from the training folder and dataset

In [ ]:
# pip install pandas

In [ ]:
import pandas as pd

df_targets = pd.read_csv('Training_set.csv')
df_targets.head()

In [ ]:
# df_targets[df_targets['filename'] == 'Image_1.jpg']['label'].values[0]

In [ ]:
# pip install opencv-python

In [ ]:
import cv2 

In [ ]:
image_data = []
image_label = []

for dirc in os.listdir(train_folder):
    file = os.path.join(train_folder, dirc)
    image = cv2.imread(file, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (250, 250), interpolation = cv2.INTER_AREA)
    image = np.array(image)
    image = image.astype('float32')
    image = image/255
    image_data.append(image)
    
    label = df_targets[df_targets['filename'] == dirc]['label'].values[0]
    image_label.append(label)

## Augmenting some random images to increase training dataset

In [ ]:
import shutil
shutil.rmtree("augmented_images")

In [ ]:
# creating directory to save augmented images

os.mkdir('augmented_images/')

In [ ]:
# pip install keras

In [ ]:
# pip install tensorflow

## Creating 400 augmented images from 100 randomly choosen images

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# pip install SciPy

In [ ]:
for dirc in np.random.choice(os.listdir(train_folder), size = 200, replace = False):
    file = os.path.join(train_folder, dirc)
    image = cv2.imread(file, cv2.COLOR_BGR2RGB)
    # image = cv2.resize(image, (250, 250), interpolation = cv2.INTER_AREA)
    
    label = df_targets[df_targets['filename'] == dirc]['label'].values[0]
    
    # Increase rank of the image by adding an axis to make it compatible for ImageDataGenerator
    # print(image.shape)
    image = image.reshape((1,) + image.shape)
    # print(image.shape)
    
    datagen = ImageDataGenerator(rotation_range = 20,
                                 width_shift_range = 0.1,
                                 height_shift_range = 0.1,
                                 zoom_range = 0.2,
                                 horizontal_flip = True,
                                 vertical_flip = True,
                                 fill_mode = 'nearest')

    i = 0
    for batch in datagen.flow(image, save_to_dir = './augmented_images', save_prefix = label, save_format = 'jpg'):
        i = i+1
        if i>1:                  # Two augmentation for each image 
            break                 

In [ ]:
# Creating zip file of augmented images folder

# shutil.make_archive('augmented_images', 'zip', '/kaggle/working/augmented_images/')

## Adding augmented images and corresponding labels to image_data and image_label

In [ ]:
aug_images_folder = './augmented_images'

for dirc in os.listdir(aug_images_folder):
    file = os.path.join(aug_images_folder, dirc)
    image = cv2.imread(file, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (250, 250), interpolation = cv2.INTER_AREA)
    image = np.array(image)
    image = image.astype('float32')
    image = image/255
    image_data.append(image)
    
    label = dirc.split('_')[0]
    image_label.append(label)

In [ ]:
print(len(image_data), len(image_label))

## Converting the object labels to numerical values

In [ ]:
dict_targets = {v : k for k,v in enumerate(np.unique(image_label))}
dict_targets

In [ ]:
# dict_targets['cloudy']

In [ ]:
targets = [dict_targets[image_label[i]] for i in range(len(image_label))]

## Construct the CNN neural network for the classification

In [ ]:
import tensorflow.keras as tf

In [ ]:
targets = tf.utils.to_categorical(targets, 5)

# or use
# np.array(list(map(int, targets)), np.float32),       
# for sparse_categorical_crossentropy loss function

In [ ]:
len(targets)

In [ ]:
model = tf.models.Sequential()

model.add(tf.layers.Conv2D(filters = 25, kernel_size = (3,3), strides = (2,2), activation = 'relu', input_shape = (250, 250, 3), padding = 'same')),
# model.add(tf.layers.BatchNormalization())
model.add(tf.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2))),

model.add(tf.layers.Conv2D(filters = 45, kernel_size = (3,3), strides = (2,2), activation = 'relu', padding = 'same')),
# model.add(tf.layers.BatchNormalization())
model.add(tf.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2))),

model.add(tf.layers.Flatten()),

model.add(tf.layers.Dense(512, activation = 'tanh')),
model.add(tf.layers.Dropout(rate = 0.2)),
model.add(tf.layers.Dense(128, activation = 'tanh')),
model.add(tf.layers.Dropout(rate = 0.1)),

model.add(tf.layers.Dense(5, activation = 'softmax'))

In [ ]:
# from tensorflow.keras.optimizers import Adam

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [ ]:
model.fit(x = np.array(image_data, np.float32),
          y = targets,
          epochs = 6)

In [ ]:
model.summary()

## Classification on the test data

In [ ]:
testing_set = pd.read_csv('Testing_set.csv')
testing_set.head()

In [ ]:
filename =  testing_set['filename'].values.tolist()
filename[:5]

In [ ]:
test_folder =  'test'

test_image_data = []
# test_image_label = []

for dirc in filename:
    file = os.path.join(test_folder, dirc)
    image = cv2.imread(file, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (250, 250), interpolation = cv2.INTER_AREA)
    image = np.array(image)
    image = image.astype('float32')
    image = image/255
    test_image_data.append(image)
    
#     label = df_test_targets[df_test_targets['filename'] == dirc]['label'].values[0]
#     test_image_label.append(label)

In [ ]:
pred = model.predict(np.array(test_image_data, np.float32))

In [ ]:
list_pred = []

for i in pred:
    list_pred.append(np.argmax(i))
    
list_pred[:5]

### Converting predicted numerical labels back to string values

In [ ]:
prediction = []
for i in list_pred:
    [prediction.append(k) for k,v in dict_targets.items() if v == i]

In [ ]:
print(len(test_image_data), len(list_pred), len(prediction)) 

In [ ]:
prediction_results = pd.DataFrame(prediction, index = filename)
prediction_results.columns = ['label']

In [ ]:
prediction_results.head()

In [ ]:
prediction_results.to_csv('CNN_prediction_results.csv', index = False)

# <span style = 'background: Red'> Pre-trained model - Huggingface </span>

## Dataset preparation

### Loading Images

In [ ]:
import pandas as pd

df_targets = pd.read_csv('Training_set.csv')
df_targets.head()

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image 

train_folder =  'train'

image_data = []
image_label = []

for dirc in os.listdir(train_folder):
    file = os.path.join(train_folder, dirc)
    
    img = image.load_img(file, target_size=(100, 100))
    img = image.img_to_array(img)
    # img = np.expand_dims(img, axis=0)
    image_data.append(img)
    
    label = df_targets[df_targets['filename'] == dirc]['label'].values[0]
    image_label.append(label)
    
# image_data = np.vstack(image_data)

# image_data.shape

In [ ]:
np.array(image_data).shape

### Creating dictionary of labels 

In [ ]:
dict_labels = {v : k for k,v in enumerate(np.unique(image_label))}
dict_labels

### Converting labels from string to numerical value

In [ ]:
image_label = [dict_labels[image_label[i]] for i in range(len(image_label))]
image_label[:5]

### Splitting image_data and image_label into train and validation 

In [ ]:
n = len(image_data)
split = int(0.80*n)

train_image_dict = {'images': image_data[:split], 'labels':image_label[:split]}
val_image_dict = {'images': image_data[split:], 'labels':image_label[split:]}

In [ ]:
# pip install datasets

### Converting data dictionary into dataset

In [ ]:
from datasets import Dataset

ds_train = Dataset.from_dict(train_image_dict)
ds_val = Dataset.from_dict(val_image_dict)

ds_train

## <span style = 'background: Yellow'> tensorflow implimentation of Hugging face model </span>

In [ ]:
#pip install transformers

In [ ]:
from transformers import TFAutoModel, AutoFeatureExtractor

### Preprocessing input dataset 

'google/vit-base-patch16-224' takes input data as pixel_values of size (color, 224, 224)

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

In [ ]:
def preprocess_images(example_batch):
    
    images = example_batch['images']

    images = [np.array(image, dtype=np.uint8) for image in images]
    
    # convert to list of NumPy arrays of shape (C, H, W)
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    
    # preprocess and add pixel_values
    inputs = feature_extractor(images=images)
    
    example_batch['pixel_values'] = inputs['pixel_values']

    return example_batch

In [ ]:
label_list = list(dict_labels.keys())
label_list

In [ ]:
# we need to define the features ourselves as both the img and pixel_values have a 3D shape 

from datasets import Features, ClassLabel, Array3D

features = Features({'images': Array3D(dtype="int64", shape=(100, 100, 3)),
                     'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
                     'labels': ClassLabel(names = label_list),
                    })

In [ ]:
# adding pixel_values feature in ds_train and ds_val by using defined preprocess_images() function for huggingface model 

preprocessed_ds_train = ds_train.map(preprocess_images, batched = True, features = features)
preprocessed_ds_val = ds_val.map(preprocess_images, batched = True, features = features)

### Convert preprocessed datasets into tensorflow datasets and collate to make batches of inpute data

In [ ]:
from transformers import DefaultDataCollator

collator = DefaultDataCollator(return_tensors = 'tf')

tf_train_dataset = preprocessed_ds_train.to_tf_dataset(columns = ['pixel_values'],
                                                       label_cols = ['labels'],
                                                       shuffle = True,
                                                       batch_size = 20,
                                                       collate_fn = collator)

tf_val_dataset = preprocessed_ds_val.to_tf_dataset(columns = ['pixel_values'],
                                                   label_cols = ['labels'],
                                                   shuffle = True,
                                                   batch_size = 20,
                                                   collate_fn = collator)

In [ ]:
tf_train_dataset

### Constructing model with pre-trained model as the non-trainable part

In [ ]:
model = TFAutoModel.from_pretrained('google/vit-base-patch16-224')

In [ ]:
model.trainable = False

In [ ]:
model.summary()

In [ ]:
import tensorflow.keras as tf

input_layer = tf.layers.Input(shape=(3, 224, 224), 
                              name='pixel_values')

In [ ]:
last_hidden_state = model(input_layer).last_hidden_state[:, 0, :]

In [ ]:
last_hidden_state

In [ ]:
output = tf.layers.Dense(5, activation = 'softmax')(last_hidden_state)

In [ ]:
model = tf.Model(input_layer, output)

In [ ]:
model.summary()

In [ ]:
model.compile(tf.optimizers.Adam(lr = 5e-3), 
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(tf_train_dataset,
         validation_data = tf_val_dataset,
         epochs = 1)

### Classification on test data

In [ ]:
testing_set = pd.read_csv('Testing_set.csv')

filename =  testing_set['filename'].values.tolist()

Testing_folder =  'test'

file_path = [Testing_folder + '/' +file for file in filename]

In [ ]:
from tensorflow.keras.preprocessing import image 

pred_image_data = []

for file in file_path:
    img = image.load_img(file, target_size=(100, 100))
    img = image.img_to_array(img)
    # img = np.expand_dims(img, axis=0)
    pred_image_data.append(img)
    
# pred_image_data = np.vstack(pred_image_data)
    
pred_image_dict = {'images': pred_image_data}

In [ ]:
ds_pred = Dataset.from_dict(pred_image_dict)

In [ ]:
preprocessed_ds_pred = ds_pred.map(preprocess_images, batched = True, batch_size = 20)

In [ ]:
preprocessed_ds_pred

In [ ]:
from transformers import DefaultDataCollator

collator = DefaultDataCollator(return_tensors = 'tf')

tf_pred_dataset = preprocessed_ds_pred.to_tf_dataset(columns = ['pixel_values'],
                                                       shuffle = True,
                                                       batch_size = 20,
                                                       collate_fn = collator
                                                      )

In [ ]:
tf_pred_dataset

In [ ]:
predictions = model.predict(tf_pred_dataset)

In [ ]:
predictions.shape

In [ ]:
predictions = np.argmax(predictions, axis = 1)

In [ ]:
labels = dict_labels
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predictions]

In [ ]:
predictions[:5]

In [ ]:
predictions = pd.DataFrame(predictions)

predictions.columns = ['label']

predictions.to_csv('tensorflow_prediction_results.csv', index = False)

## <span style = 'background: Yellow'> pytorch implimentation of Hugging face model </span>

### Preprocessing input dataset 

'google/vit-base-patch16-224' takes input data as pixel_values of size (color, 224, 224)

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

In [ ]:
def preprocess_images(example_batch):
    
    images = example_batch['images']
    
    images = [np.array(image, dtype=np.uint8) for image in images]
    
    # convert to list of NumPy arrays of shape (C, H, W)
    images = [np.moveaxis(image, source=-1, destination=0) for image in images]
    
    # preprocess and add pixel_values
    inputs = feature_extractor(images=images)
    example_batch['pixel_values'] = inputs['pixel_values']

    return example_batch

In [ ]:
label_list = list(dict_labels.keys())
label_list

In [ ]:
# we need to define the features ourselves as both the img and pixel_values have a 3D shape

from datasets import Features, ClassLabel, Array3D
 
features = Features({'images': Array3D(dtype="int64", shape=(100, 100, 3)),
                     'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
                     'labels': ClassLabel(names = label_list)
                    })

In [ ]:
preprocessed_ds_train = ds_train.map(preprocess_images, batched = True, batch_size = 20, features = features)
preprocessed_ds_val = ds_val.map(preprocess_images, batched = True, batch_size = 20, features = features)

In [ ]:
preprocessed_ds_train

In [ ]:
preprocessed_ds_train.info.features

In [ ]:
preprocessed_ds_train.features['labels'].names

### Constructing model with pre-trained model as the non-trainable part

In [ ]:
# pip install torch

In [ ]:
from transformers import ViTModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn

class ViTForImageClassification(nn.Module):
    
    def __init__(self):
        super(ViTForImageClassification, self).__init__()
        self.model = ViTModel.from_pretrained("google/vit-base-patch16-224")
        
        # Freezing layers of ViTModel
        for lay in self.model.parameters():
            lay.requires_grad = False
    
        self.linear = nn.Linear(self.model.config.hidden_size, 5)
        self.softmax = nn.Softmax()
        
    def forward(self, pixel_values, labels = None):
        outputs = self.model(pixel_values = pixel_values)
        output = outputs.last_hidden_state[:, 0, :]
        logits = self.linear(output)
        
        loss = None
        if labels is not None:
            loss_function = nn.CrossEntropyLoss()
            loss = loss_function(logits, labels)
        
        return SequenceClassifierOutput(loss = loss, logits = logits)

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir = "./pytorch_results",
                         evaluation_strategy = "epoch",
                         learning_rate = 5e-3,
                         num_train_epochs = 1,
                         per_device_train_batch_size = 20,
                         per_device_eval_batch_size = 20,
                         gradient_accumulation_steps = 1,
                        )

# args = TrainingArguments(output_dir = "./pytorch_results",
#                          optim = 'adamw_torch',
#                          evaluation_strategy = "steps",
#                          save_strategy = "steps",
#                          save_steps = 10,
#                          eval_steps = 10,
#                          learning_rate = 5e-3,
#                          num_train_epochs = 1,
#                          per_device_train_batch_size = 20,
#                          per_device_eval_batch_size = 20,
#                          gradient_accumulation_steps = 1,
#                          weight_decay = 0.01,
#                          load_best_model_at_end=True,
#                          metric_for_best_model = metric_name,
#                          logging_dir = './pytorch_results/logs',
#                         )

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
# pip uninstall torchvision torchaudio

In [ ]:
model = ViTForImageClassification()

In [ ]:
# Number of trainable parameters in model

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

In [ ]:
# pip install sklearn

In [ ]:
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions = predictions, references = labels)

In [ ]:
trainer = Trainer(model,
                  args,
                  train_dataset = preprocessed_ds_train,
                  eval_dataset = preprocessed_ds_val,
                  data_collator = data_collator,
                  compute_metrics = compute_metrics
                 )

In [ ]:
%%time

trainer.train()

### Classification on test data

In [ ]:
testing_set = pd.read_csv('Testing_set.csv')

filename =  testing_set['filename'].values.tolist()

Testing_folder =  'test'

file_path = [Testing_folder + '/' +file for file in filename]

In [ ]:
from tensorflow.keras.preprocessing import image 

pred_image_data = []

for file in file_path:
    img = image.load_img(file, target_size=(100, 100))
    img = image.img_to_array(img)
    # img = np.expand_dims(img, axis=0)
    pred_image_data.append(img)
    
# pred_image_data = np.vstack(pred_image_data)
    
pred_image_dict = {'images': pred_image_data}

In [ ]:
ds_pred = Dataset.from_dict(pred_image_dict)

In [ ]:
ds_pred

In [ ]:
preprocessed_ds_pred = ds_pred.map(preprocess_images, batched = True, batch_size = 20)

In [ ]:
preprocessed_ds_pred

In [ ]:
import torch

with torch.no_grad():
    outputs = trainer.predict(preprocessed_ds_pred)

In [ ]:
predictions = [np.argmax(x) for x in outputs.predictions]

In [ ]:
labels = dict_labels
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predictions]

In [ ]:
predictions[:5]

In [ ]:
predictions = pd.DataFrame(predictions)

predictions.columns = ['label']

predictions.to_csv('pytorch_prediction_results.csv', index = False)

# <span style = 'background: Red'> Pre-trained models - from tf.keras.application </span>

## Loading images path and label into a dataframe

In [ ]:
import pandas as pd

df_targets = pd.read_csv('Training_set.csv')

import os

train_folder =  'train'
file_path = []
labels = []

for dirc in os.listdir(train_folder):
    file = os.path.join(train_folder, dirc)
    file_path.append(file)
    
    label = df_targets[df_targets['filename'] == dirc]['label'].values[0]
    labels.append(label)
    
image_data = pd.DataFrame({'file_path': file_path, 'labels': labels})

image_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(image_data, test_size = 0.2, random_state = 6)

## Generating and preprocessing input dataset

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

VGG16_preprocess = preprocess_input

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(preprocessing_function = VGG16_preprocess)\
                                   .flow_from_dataframe(dataframe = train_df, 
                                                        x_col = 'file_path', 
                                                        y_col = 'labels', 
                                                        target_size = (224, 224), 
                                                        batch_size = 20, 
                                                        seed = 6
                                                        )
val_datagen = ImageDataGenerator(preprocessing_function = VGG16_preprocess)\
                                   .flow_from_dataframe(dataframe = val_df, 
                                                        x_col = 'file_path', 
                                                        y_col = 'labels', 
                                                        target_size = (224, 224), 
                                                        batch_size = 20, 
                                                        seed = 6,
                                                        )

In [ ]:
imgs, labels = next(train_datagen)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 20, figsize = (20, 20))

for img, ax in zip(imgs, axes):
    ax.imshow(img)
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# def image_generation(pre, train):
#     train_datagen = ImageDataGenerator(preprocessing_function = pre, validation_split = 0.15)
    
#     train_gen = train_datagen.flow_from_dataframe(
#                                                     dataframe=train,
#                                                     x_col='file_path',
#                                                     y_col='labels',
#                                                     target_size=(100,100),
#                                                     class_mode='categorical',
#                                                     batch_size=32,
#                                                     shuffle=True,
#                                                     seed=0,
        
#                                                     subset='training',
        
#                                                     rotation_range=30,
#                                                     zoom_range=0.15,
#                                                     width_shift_range=0.2,
#                                                     height_shift_range=0.2,
#                                                     shear_range=0.15,
#                                                     horizontal_flip=True,
#                                                     fill_mode="nearest"
#                                                 ) 
    
#     valid_gen = train_datagen.flow_from_dataframe(
#                                                     dataframe=train,
#                                                     x_col='file_path',
#                                                     y_col='labels',
#                                                     target_size=(100,100),
#                                                     class_mode='categorical',
#                                                     batch_size=32,
#                                                     shuffle=False,
#                                                     seed=0,
        
#                                                     subset='validation',
        
#                                                     rotation_range=30,
#                                                     zoom_range=0.15,
#                                                     width_shift_range=0.2,
#                                                     height_shift_range=0.2,
#                                                     shear_range=0.15,
#                                                     horizontal_flip=True,
#                                                     fill_mode="nearest"
#                                                 )
    
#     return train_gen, valid_gen

## Constructing model with pre-trained model as the non-trainable part

In [ ]:
from tensorflow.keras.applications import VGG16

vgg16 = VGG16(include_top = False, pooling = 'avg', input_shape = (224, 224, 3))

In [ ]:
type(vgg16)

In [ ]:
vgg16.trainable = False

vgg16.summary()

In [ ]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import models

model = models.Sequential()
model.add(vgg16)
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping

# callbacks  = [EarlyStopping(monitor = 'val_loss',
#                             min_delta = 0,
#                             patience = 2,
#                             mode = 'auto'
#                             )]

In [ ]:
model.fit(x = train_datagen,
          validation_data = val_datagen, 
          epochs = 1, 
          # steps_per_epoch = 42, 
          # callbacks = callbacks,
          verbose = 2
         )

In [ ]:
# from tensorflow.keras.applications import VGG19
# from tensorflow.keras.applications.vgg19 import preprocess_input

# VGG19_preprocess = preprocess_input

# train_gen_VGG19, valid_gen_VGG19 = image_generation(VGG19_preprocess, train_df)

## Classification on test data

In [ ]:
testing_set = pd.read_csv('Testing_set.csv')

filename =  testing_set['filename'].values.tolist()

Testing_folder =  'test'
file_path = []

for dirc in filename:
    file = os.path.join(Testing_folder, dirc)
    file_path.append(file)
    
pred_image_data = pd.DataFrame({'file_path': file_path})

In [ ]:
pred_datagen = ImageDataGenerator(preprocessing_function = VGG16_preprocess)\
                                   .flow_from_dataframe(dataframe = pred_image_data, 
                                                        x_col = 'file_path',  
                                                        class_mode = None,
                                                        target_size = (224, 224), 
                                                        batch_size = 20, 
                                                        seed = 6
                                                        )

In [ ]:
import numpy as np 

predictions = model.predict(pred_datagen)
predictions = np.argmax(predictions, axis = 1)

In [ ]:
predictions[:5]

In [ ]:
labels = (train_datagen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predictions]

In [ ]:
predictions[:5]

In [ ]:
predictions = pd.DataFrame(predictions)

predictions.columns = ['label']

predictions.to_csv('tf_keras_api_prediction_results.csv', index = False)

In [ ]:
predictions = pd.DataFrame(predictions)

predictions.columns = ['label']

predictions.to_csv('prediction_results.csv', index = False)